In [23]:
import requests
import json
import pandas as pd

In [4]:
#通过12306网站搜索时network中显示的网址，能够获取目标数据
url = 'https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date=2020-04-25&leftTicketDTO.from_station=SHH&leftTicketDTO.to_station=NJH&purpose_codes=ADULT'

In [11]:
#发现直接有requests无法获取到信息，构建一个headers
headers={
    "Accept": "text/html, application/xhtml+xml, application/xml; q=0.9, */*; q=0.8",
    "Connection": "Keep-Alive",
    "Cookie": "JSESSIONID=CE1E0910299807B0B5D6CE1AE4B37318; RAIL_DEVICEID=KIGorQUspNUQqHtpHqI4qcn_7n8neHBX58xwMxWqPc26JQIimw-uxozqOZEOb_cizOpzFiNc04ZF_sK3raRaFjH3Ao-IO7KIEkB24NnWaD_4ME1YfS8KrNhz637Z2NQIjcjzvKENUq6HfF0Thy8UHfZpscWtyFJf; RAIL_EXPIRATION=1587115346591; route=6f50b51faa11b987e576cdb301e545c4; BIGipServerotn=1490616586.50210.0000; _jc_save_toStation=%u5170%u5DDE%2CLZJ; _jc_save_wfdc_flag=dc; _jc_save_toDate=2020-04-14; _jc_save_fromDate=2020-04-14; _jc_save_fromStation=%u897F%u5B89%2CXAY",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362"
        }

In [13]:
js = requests.get(url,headers=headers).text

In [14]:
#将数据转换为json格式
js2 =  json.loads(js)

In [18]:
#获取目标数据
data = js2['data']['result']

In [29]:
#将数据分行分列存入列表中
dt_all = []
for dt in data:
    dt_all.append(dt.split('|'))

In [33]:
#数据是列表套列表的二维数组，导入pandas
df_use = pd.DataFrame(dt_all)

In [34]:
#过滤掉不是高铁的行
df_use = df_use[df_use[3].str.contains('D') | df_use[3].str.contains('G')]

In [35]:
#取出有用的列
df_use = df_use[[2,3,6,7,8,9,22,26,30,31,32,33]]

In [36]:
#更改列名
df_use = df_use.rename(columns={2:'编号',3:'车次',6:'发车站',7:'到达站',
                                8:'发车时间',9:'到达时间',22:'其他',26:'无座',
                                30:'二等座',31:'一等座',32:'商务特等座',33:'动卧'})

In [37]:
df_use.head()

,编号,车次,发车站,到达站,发车时间,到达时间,其他,无座,二等座,一等座,商务特等座,动卧
2,55000G7072B0,G7072,SHH,NKH,05:48,07:50,,无,有,20,5,
3,55000G703260,G7032,SHH,NJH,05:53,07:53,,无,有,有,9,
4,5l0000G45650,G456,AOH,NKH,06:05,07:32,,,有,有,14,
5,5l000G197050,G1970,AOH,NKH,24:00,24:00,,,,,,
6,5l0000D352A0,D352,AOH,NKH,06:13,08:25,,无,有,有,,


In [41]:
#空数据填充0值
df_use.replace('',0,inplace=True)

In [43]:
df_use.to_csv(r'../../file/subject/sj2_data.csv')

#### 代码整理

In [62]:
import requests
import json
import pandas

def get_pd(url,headers):
    text = requests.get(url,headers=headers).text
    return text

def to_csv(text):
    js = json.loads(text)
    data = js['data']['result']
    dt_all = []
    for dt in data:
        dt_all.append(dt.split('|'))
    
    df_use = pd.DataFrame(dt_all)
    df_use = df_use[df_use[3].str.contains('D') | df_use[3].str.contains('G')]
    df_use = df_use[[2,3,6,7,8,9,22,26,30,31,32,33]]
    df_use = df_use.rename(columns={2:'编号',3:'车次',6:'发车站',7:'到达站',
                                8:'发车时间',9:'到达时间',22:'其他',26:'无座',
                                30:'二等座',31:'一等座',32:'商务特等座',33:'动卧'})
    df_use.replace('',0,inplace=True)
    df_use.to_csv(r'../../file/subject/sj2_data01.csv')
    
if __name__ == '__main__':
    headers={
    "Accept": "text/html, application/xhtml+xml, application/xml; q=0.9, */*; q=0.8",
    "Connection": "Keep-Alive",
    "Cookie": "JSESSIONID=CE1E0910299807B0B5D6CE1AE4B37318; RAIL_DEVICEID=KIGorQUspNUQqHtpHqI4qcn_7n8neHBX58xwMxWqPc26JQIimw-uxozqOZEOb_cizOpzFiNc04ZF_sK3raRaFjH3Ao-IO7KIEkB24NnWaD_4ME1YfS8KrNhz637Z2NQIjcjzvKENUq6HfF0Thy8UHfZpscWtyFJf; RAIL_EXPIRATION=1587115346591; route=6f50b51faa11b987e576cdb301e545c4; BIGipServerotn=1490616586.50210.0000; _jc_save_toStation=%u5170%u5DDE%2CLZJ; _jc_save_wfdc_flag=dc; _jc_save_toDate=2020-04-14; _jc_save_fromDate=2020-04-14; _jc_save_fromStation=%u897F%u5B89%2CXAY",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362"
        }
    url = 'https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date=2020-04-25&leftTicketDTO.from_station=SHH&leftTicketDTO.to_station=NJH&purpose_codes=ADULT'
    text = get_pd(url,headers)
    to_csv(text)
    print('finished!')

finished!
